# Tutorial 06, case 7b: Stokes problem with Neumann control

In this tutorial we solve the optimal control problem

$$\min J(y, u) = \frac{1}{2} \int_{\Omega} (v - v_d)^2 dx + \frac{\alpha_1}{2} \int_{\Gamma_C} |\partial_t u|^2 ds + \frac{\alpha_2}{2} \int_{\Gamma_C} |u|^2 ds$$
s.t.
$$\begin{cases}
- \nu \Delta v + \nabla p = f       & \text{in } \Omega\\
             \text{div} v = 0       & \text{in } \Omega\\
                        v = g       & \text{on } \Gamma_{in}\\
                        v = 0       & \text{on } \Gamma_{w}\\
   p n - \nu \partial_n v = u       & \text{on } \Gamma_{N}
\end{cases}$$

where
$$\begin{align*}
& \Omega                      & \text{unit square}\\
& \Gamma_{in}                 & \text{has boundary id 1}\\
& \Gamma_{w}                  & \text{has boundary id 2}\\
& \Gamma_{N}                  & \text{has boundary id 3}\\
& \Gamma_{C}                  & \text{has boundary id 4}\\
& u \in [L^2(\Gamma_C)]^2     & \text{control variable}\\
& v \in [H^1_0(\Omega)]^2     & \text{state velocity variable}\\
& p \in L^2(\Omega)           & \text{state pressure variable}\\
& \alpha_1, \alpha_2 > 0      & \text{penalization parameters}\\
& v_d                         & \text{desired state}\\
& f                           & \text{forcing term}\\
& g                           & \text{inlet profile}\\
\end{align*}$$
using an adjoint formulation solved by a one shot approach

In [ ]:
from numpy import arange, isclose, isin, where, zeros
from ufl import *
from dolfinx import *
from dolfinx.cpp.mesh import GhostMode
from dolfinx.fem import assemble_scalar, locate_dofs_topological
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot
from multiphenics import *

### Mesh

In [ ]:
if MPI.size(MPI.comm_world) > 1:
    mesh_ghost_mode = GhostMode.shared_facet # shared_facet ghost mode is required by dS
else:
    mesh_ghost_mode = GhostMode.none
mesh = XDMFFile(MPI.comm_world, "data/bifurcation.xdmf").read_mesh(mesh_ghost_mode)
subdomains = XDMFFile(MPI.comm_world, "data/bifurcation_subdomains.xdmf").read_mf_size_t(mesh)
boundaries = XDMFFile(MPI.comm_world, "data/bifurcation_boundaries.xdmf").read_mf_size_t(mesh)
boundaries_1 = where(boundaries.values == 1)[0]
boundaries_2 = where(boundaries.values == 2)[0]
boundaries_4 = where(boundaries.values == 4)[0]
boundaries_12 = where(isin(boundaries.values, (1, 2)))[0]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)
dS = Measure("dS")(subdomain_data=boundaries)

In [ ]:
# Normal and tangent
n = FacetNormal(mesh)
t = as_vector([n[1], -n[0]])

### Function spaces

In [ ]:
Y_velocity = VectorFunctionSpace(mesh, ("Lagrange", 2))
Y_pressure = FunctionSpace(mesh, ("Lagrange", 1))
U = VectorFunctionSpace(mesh, ("Lagrange", 2))
Q_velocity = Y_velocity
Q_pressure = Y_pressure
dofs_Y_velocity = arange(0, Y_velocity.dofmap.index_map.block_size*(Y_velocity.dofmap.index_map.size_local + Y_velocity.dofmap.index_map.num_ghosts))
dofs_Y_pressure = arange(0, Y_pressure.dofmap.index_map.block_size*(Y_pressure.dofmap.index_map.size_local + Y_pressure.dofmap.index_map.num_ghosts))
dofs_U = locate_dofs_topological(U, boundaries.dim, boundaries_4)
dofs_Q_velocity = dofs_Y_velocity
dofs_Q_pressure = dofs_Y_pressure
W = BlockFunctionSpace([Y_velocity, Y_pressure, U, Q_velocity, Q_pressure],
                       restrict=[dofs_Y_velocity, dofs_Y_pressure, dofs_U, dofs_Q_velocity, dofs_Q_pressure])

In [ ]:
trial = BlockTrialFunction(W)
(v, p, u, z, b) = block_split(trial)
test = BlockTestFunction(W)
(w, q, r, s, d) = block_split(test)

 ### Problem data

In [ ]:
nu = 0.04
alpha_1 = 0.001
alpha_2 = 0.1*alpha_1
x = SpatialCoordinate(mesh)
a = 1.0
b = 0.8
v_d = as_vector((a*(b*10.0*(x[1]**3 - x[1]**2 - x[1] + 1.0)) + ((1.0-b)*10.0*(-x[1]**3 - x[1]**2 + x[1] + 1.0)), 0.0))
f = Constant(mesh, (0., 0.))
def g_eval(x):
    values = zeros((2, x.shape[1]))
    values[0, :] = 10.0*a*(x[1, :] + 1.0)*(1.0 - x[1, :])
    return values
g = Function(W.sub(0))
g.interpolate(g_eval)
bc0 = Function(W.sub(0))

### Optimality conditions

In [ ]:
def tracking(v, w):
    return inner(v, w)("-")
def penalty(u, r):
    return alpha_1*inner(grad(u)*t, grad(r)*t) + alpha_2*inner(u, r)
a = [[tracking(v, w)*dS(4)         , 0            , 0                  , nu*inner(grad(z), grad(w))*dx, - b*div(w)*dx],
     [0                            , 0            , 0                  , - q*div(z)*dx                , 0            ],
     [0                            , 0            , penalty(u, r)*ds(3), - inner(z, r)*ds(3)          , 0            ],
     [nu*inner(grad(v), grad(s))*dx, - p*div(s)*dx, - inner(u, s)*ds(3), 0                            , 0            ],
     [- d*div(v)*dx                , 0            , 0                  , 0                            , 0            ]]
f =  [tracking(v_d, w)*dS(4),
      0                     ,
      0                     ,
      inner(f, s)*dx        ,
      0                      ]
bdofs_W0_1 = locate_dofs_topological((W.sub(0), W.sub(0)), mesh.topology.dim - 1, boundaries_1)
bdofs_W0_2 = locate_dofs_topological((W.sub(0), W.sub(0)), mesh.topology.dim - 1, boundaries_2)
bdofs_W3_12 = locate_dofs_topological((W.sub(3), W.sub(0)), mesh.topology.dim - 1, boundaries_12)
bc = BlockDirichletBC([[DirichletBC(g, bdofs_W0_1, W.sub(0)), DirichletBC(bc0, bdofs_W0_2, W.sub(0))],
                       [],
                       [],
                       [DirichletBC(bc0, bdofs_W3_12, W.sub(3))],
                       []], W)

### Solution

In [ ]:
solution = BlockFunction(W)
(v, p, u, z, b) = block_split(solution)

### Cost functional

In [ ]:
J = 0.5*tracking(v - v_d, v - v_d)*dS(4) + 0.5*penalty(u, u)*ds(3)

### Uncontrolled functional value

In [ ]:
W_state_trial = W.extract_block_sub_space((0, 1))
W_state_test = W.extract_block_sub_space((3, 4))

In [ ]:
a_state = block_restrict(a, [W_state_test, W_state_trial])
f_state = block_restrict(f, W_state_test)
bc_state = block_restrict(bc, W_state_trial)

In [ ]:
solution_state = block_restrict(solution, W_state_trial)

In [ ]:
solver_parameters = {"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"}
block_solve(a_state, solution_state, f_state, bc_state, petsc_options=solver_parameters)

In [ ]:
J_uncontrolled = MPI.sum(mesh.mpi_comm(), assemble_scalar(J))
print("Uncontrolled J =", J_uncontrolled)
assert isclose(J_uncontrolled, 2.8479865)

In [ ]:
plot(v, title="uncontrolled state velocity")

In [ ]:
plot(p, title="uncontrolled state pressure")

### Optimal control

In [ ]:
block_solve(a, solution, f, bc, petsc_options=solver_parameters)

In [ ]:
J_controlled = MPI.sum(mesh.mpi_comm(), assemble_scalar(J))
print("Optimal J =", J_controlled)
assert isclose(J_controlled, 1.7643950)

In [ ]:
plot(v, title="state velocity")

In [ ]:
plot(p, title="state pressure")

In [ ]:
plot(u, title="control")

In [ ]:
plot(z, title="adjoint velocity")

In [ ]:
plot(b, title="adjoint pressure")